Test 

In [1]:
import tensorflow as tf
from define import *
import wfdb
import numpy as np
from scipy.signal import resample
from preprocessing import *
from util import *
from make_data import *
import pandas as pd
import os

In [2]:
def get_ppg_model(input_shape=NEIGHBOUR_POINT, learning_rate=0.02, momentum=0.9):
    cnn_model = tf.keras.models.Sequential()
    cnn_model.add(tf.keras.layers.Conv1D(filters=8, kernel_size=5, padding='same', activation='relu',
                                         input_shape=(input_shape, 1)))
    cnn_model.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'))
    cnn_model.add(tf.keras.layers.Conv1D(filters=16, kernel_size=5, padding='same', activation='relu'))
    cnn_model.add(tf.keras.layers.Flatten())
    cnn_model.add(tf.keras.layers.Dense(32, activation='relu'))
    cnn_model.add(tf.keras.layers.Dense(2, activation='softmax'))

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    loss = tf.keras.losses.binary_crossentropy
    cnn_model.compile(optimizer, loss=loss, metrics=['accuracy'])
    return cnn_model


In [ ]:
def predict_on_csv(file_path, weights_file_path, ppg_column_name):
    """
    Hàm này tải, xử lý, và dự đoán BPM từ một file CSV.
    """
    
    # --- 1. Tải và chuẩn bị dữ liệu (Load and Prep Data) ---
    print(f"Đang đọc file: {file_path}...")
    try:
        df = pd.read_csv(file_path)
        signal = df[ppg_column_name].values.astype('float32')
        original_fs = 125
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy file tại '{file_path}'")
        return None, None
    except KeyError:
        print(f"LỖI: Không tìm thấy cột '{ppg_column_name}' trong file CSV.")
        print(f"Các cột có sẵn là: {df.columns.tolist()}")
        return None, None
    except Exception as e:
        print(f"Lỗi khi đọc CSV: {e}")
        return None, None

    # --- 2. Tiền xử lý (Preprocess) ---
    print(f"Tần số gốc: {original_fs}Hz. Tần số mục tiêu: {FREQUENCY_SAMPLING}Hz.")
    if original_fs != FREQUENCY_SAMPLING:
        # Resample tín hiệu về 125Hz
        new_length = int(len(signal) * FREQUENCY_SAMPLING / original_fs)
        signal = resample(signal, new_length)
        print(f"Đã resample tín hiệu. Độ dài mới: {new_length} mẫu.")
    else:
        print("Tần số gốc đã khớp, không cần resample.")

    # Áp dụng 2 hàm tiền xử lý giống hệt lúc train
    signal_processed = baseline_wander_remove(signal, FREQUENCY_SAMPLING)
    signal_processed = normalize(signal_processed, FREQUENCY_SAMPLING)
    print("Đã áp dụng baseline_wander_remove và normalize.")

    # --- 3. Tạo cửa sổ (Create Windows) ---
    data_sample = []
    window_size = NEIGHBOUR_POINT # NEIGHBOUR_POINT = 50
    
    # Lặp để tạo các cửa sổ chồng lấn
    for i in range(len(signal_processed) - (window_size - 1)):
        data_sample.append(signal_processed[i : i + window_size])
    
    if not data_sample:
        print(f"Tín hiệu quá ngắn (dài {len(signal_processed)} mẫu) để tạo cửa sổ (kích thước {window_size}).")
        return None, None
        
    # Chuyển sang định dạng model mong muốn (N, 50, 1)
    data_sample_np = np.expand_dims(np.array(data_sample, dtype='float32'), axis=2)
    print(f"Đã tạo {data_sample_np.shape[0]} cửa sổ dữ liệu.")

    # --- 4. Tải Model và Trọng số (Load Model) ---
    try:
        # Gọi hàm từ cell [11b93ad1]
        model = tf.keras.models.load_model("last_ckt.weights.h5")
        print(f"Đã tải trọng số từ {weights_file_path}")
    except NameError:
        print("LỖI: Hàm `get_ppg_model()` chưa được định nghĩa.")
        print("Hãy chạy cell [11b93ad1] trước.")
        return None, None
    except Exception as e:
        print(f"LỖI khi tải trọng số: {e}")
        print("Hãy kiểm tra lại đường dẫn `weights_file_path`.")
        return None, None

    # --- 5. Dự đoán (Predict) ---
    print("Đang chạy dự đoán (model.predict)...")
    prediction_probs = model.predict(data_sample_np, batch_size=128, verbose=0)
    print("Đã chạy dự đoán xong.")

    # --- 6. Hậu xử lý (Post-process) ---
    # Lấy xác suất là đỉnh (cột 1)
    prediction_rounded = np.rint(prediction_probs)
    predicted_class_1 = prediction_rounded[:, 1]
    
    # Nhóm các đỉnh lại và bù offset
    predicted_peaks = clustering(predicted_class_1) + int(0.1 * FREQUENCY_SAMPLING)
    print(f"Phát hiện được {len(predicted_peaks)} đỉnh.")

    # --- 7. Tính BPM (Calculate BPM) ---
    bpm_pred = calculate_average_bpm(predicted_peaks, FREQUENCY_SAMPLING) #
    
    return predicted_peaks, bpm_pred

In [5]:
my_weights_path = r'D:\altium and source projects\Health monitoring\AI_trainning\qrs_detection_dataset\checkpoint\mitdb\run-0\09.weights.h5'
my_csv_path = 'ppg_test_7500_noisy.csv'
my_ppg_column = 'IR'  # Dựa trên file 1.csv, chọn 'Red (a.u)' hoặc 'Infra Red (a.u)'
# 2. GỌI HÀM ĐỂ CHẠY
print("--- BẮT ĐẦU DỰ ĐOÁN ---")
peaks, bpm = predict_on_csv(my_csv_path, 
                           my_weights_path, 
                           my_ppg_column, 
                           )

if peaks is not None:
    print("\n--- ✅ KẾT QUẢ CUỐI CÙNG ---")
    print(f"Đã tìm thấy các đỉnh tại mẫu (sau resample): {peaks}")
    print(f"BPM dự đoán trung bình: {bpm:.2f} BPM")
else:
    print("\n--- ❌ ĐÃ XẢY RA LỖI ---")
    print("Quá trình dự đoán thất bại. Vui lòng xem lại thông báo lỗi ở trên.")

--- BẮT ĐẦU DỰ ĐOÁN ---
Đang đọc file: ppg_test_7500_noisy.csv...
Tần số gốc: 125Hz. Tần số mục tiêu: 125Hz.
Tần số gốc đã khớp, không cần resample.
Đã áp dụng baseline_wander_remove và normalize.
Đã tạo 7451 cửa sổ dữ liệu.


Đã tải trọng số từ D:\altium and source projects\Health monitoring\AI_trainning\qrs_detection_dataset\checkpoint\mitdb\run-0\09.weights.h5
Đang chạy dự đoán (model.predict)...
Đã chạy dự đoán xong.
Phát hiện được 76 đỉnh.

--- ✅ KẾT QUẢ CUỐI CÙNG ---
Đã tìm thấy các đỉnh tại mẫu (sau resample): [  25  117  230  330  435  534  647  750  846  960 1063 1271 1365 1386
 1466 1577 1678 1707 1789 1905 1999 2095 2209 2309 2408 2511 2622 2728
 2841 2932 3040 3163 3247 3350 3455 3558 3668 3771 3869 3977 4077 4184
 4289 4397 4499 4598 4701 4721 4814 4913 5010 5117 5216 5240 5324 5426
 5451 5537 5646 5752 5967 6062 6167 6276 6363 6389 6468 6575 6685 6784
 6894 6998 7100 7209 7311 7411]
BPM dự đoán trung bình: 74.30 BPM


In [ ]:
import tensorflow as tf

# load model keras
model = tf.keras.models.load_model("last_ckt.weights.h5")

# convert sang tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# bật quantization (int8) để chạy được trên MCU
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)


In [2]:
import os

# Tên file input/output
input_file = "model.tflite"
cc_file = "model_data.cc"
h_file = "model_data.h"
array_name = "model_tflite"

# Đọc file .tflite
with open(input_file, "rb") as f:
    data = f.read()

# Sinh file .cc
with open(cc_file, "w") as f:
    f.write(f'#include "{h_file}"\n\n')
    f.write(f"const unsigned char {array_name}[] = {{\n")
    for i in range(0, len(data), 12):
        chunk = data[i:i+12]
        f.write("  " + ", ".join(f"0x{b:02x}" for b in chunk) + ",\n")
    f.write("};\n\n")
    f.write(f"const unsigned int {array_name}_len = {len(data)};\n")

# Sinh file .h
with open(h_file, "w") as f:
    f.write("#pragma once\n\n")
    f.write("#include <cstdint>\n\n")
    f.write(f"extern const unsigned char {array_name}[];\n")
    f.write(f"extern const unsigned int {array_name}_len;\n")

print(f"✅ Đã tạo {cc_file} và {h_file}, dung lượng model = {len(data)} bytes")


✅ Đã tạo model_data.cc và model_data.h, dung lượng model = 21880 bytes
